In [ ]:
%%writefile 2Dred.cu

#include <iostream>         // For standard input/output operations
#include <cuda_runtime.h>   // For CUDA runtime API
#include <device_launch_parameters.h> // Optional but ensures threadIdx, blockIdx, etc., are defined. mostly included with cuda_runtime.h
#include <stdlib.h>

#define num_threads 512

__global__ void reduce2D(float* src, float* dst, int rows, int cols){

  int index = threadIdx.x;

//Each thread mapped to an individual row.

  for (int i=index; i<rows; i+=num_threads){
    float sum= 0.0f;
    for (int j=0; j<cols; j++) {
      sum+=src[i*cols+j];
    }
    dst[i]=sum;
  }

}

int main(){
  int rows=10000, cols=10000;
  size_t size= rows*cols*sizeof(float);
  float* h_src= new float[rows*cols];
  float* h_dst = new float[rows];

  for (int i=0; i<rows*cols; i++){
    h_src[i] = 1.0f;
  }

  float* d_src, *d_dst;
  cudaMalloc(&d_src,size);
  cudaMalloc(&d_dst, rows*sizeof(float));

  cudaMemcpy(d_src,h_src,size,cudaMemcpyHostToDevice);

  reduce2D<<<1,num_threads>>>(d_src,d_dst,rows,cols);

  cudaDeviceSynchronize();

  cudaMemcpy(h_dst,d_dst, rows*sizeof(float),cudaMemcpyDeviceToHost);

  for (int i=0; i<20; i++){
    std::cout << h_dst[i] << " ";
  }

  return 0;


}

Writing 2Dred.cu


In [ ]:
!nvcc -gencode=arch=compute_75,code=sm_75 -o reduction 2Dred.cu

In [ ]:
%%shell
nvprof ./reduction

==402== NVPROF is profiling process 402, command: ./reduction
10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ==402== Profiling application: ./reduction
==402== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   60.12%  145.78ms         1  145.78ms  145.78ms  145.78ms  reduce2D(float*, float*, int, int)
                   39.88%  96.695ms         1  96.695ms  96.695ms  96.695ms  [CUDA memcpy HtoD]
                    0.00%  5.5040us         1  5.5040us  5.5040us  5.5040us  [CUDA memcpy DtoH]
      API calls:   37.60%  147.97ms         2  73.985ms  117.09us  147.85ms  cudaMalloc
                   37.05%  145.79ms         1  145.79ms  145.79ms  145.79ms  cudaDeviceSynchronize
                   24.92%  98.072ms         2  49.036ms  80.804us  97.992ms  cudaMemcpy
                    0.29%  1.1425ms         1  1.1425ms  1.1425ms  1.1425ms  cuDeviceGet

In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [ ]:
!nvidia-smi

Sat Mar 22 08:22:27 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   63C    P8             13W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!apt install -y nsight-compute-2025.1.1

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  nsight-compute-2025.1.1
0 upgraded, 1 newly installed, 0 to remove and 30 not upgraded.
Need to get 295 MB of archives.
After this operation, 1,195 MB of additional disk space will be used.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  nsight-compute-2025.1.1 2025.1.1.2-1 [295 MB]
Fetched 295 MB in 4s (74.5 MB/s)
Selecting previously unselected package nsight-compute-2025.1.1.
(Reading database ... 126209 files and directories currently installed.)
Preparing to unpack .../nsight-compute-2025.1.1_2025.1.1.2-1_amd64.deb ...
Unpacking nsight-compute-2025.1.1 (2025.1.1.2-1) ...
Setting up nsight-compute-2025.1.1 (2025.1.1.2-1) ...
Processing triggers for mailcap (3.70+nmu1ubuntu1) ...


In [ ]:
!ncu --version

NVIDIA (R) Nsight Compute Command Line Profiler
Copyright (c) 2018-2024 NVIDIA Corporation
Version 2024.2.1.0 (build 34372528) (public-release)


In [ ]:
!ncu ./reduction

==PROF== Connected to process 6768 (/content/reduction)
==PROF== Profiling "reduce2D" - 0: 0%....50%....100% - 9 passes
1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 ==PROF== Disconnected from process 6768
[6768] reduction@127.0.0.1
  reduce2D(float *, float *, int, int) (1, 1, 1)x(512, 1, 1), Context 1, Stream 7, Device 0, CC 7.5
    Section: GPU Speed Of Light Throughput
    ----------------------- ----------- ------------
    Metric Name             Metric Unit Metric Value
    ----------------------- ----------- ------------
    DRAM Frequency                  Ghz         5.00
    SM Frequency                    Mhz       584.97
    Elapsed Cycles                cycle      765,119
    Memory Throughput                 %         2.20
    DRAM Throughput                   %         2.20
    Duration                         ms         1.31
    L1/TEX Cache Throughput           %        83.12
    L2 Cache Throughput               % 

In [ ]:
!ncu --help

usage: ncu [options] [program] [program-arguments]

General Options:
  -h [ --help ]                         Print this help message.
  -v [ --version ]                      Print the version number.
  --mode arg (=launch-and-attach)       Select the mode of interaction with the target application:
                                          launch-and-attach
                                          (launch and attach for profiling)
                                          launch
                                          (launch and suspend for later attach)
                                          attach
                                          (attach to launched application)
  -p [ --port ] arg (=49152)            Base port for connecting to target application
  --max-connections arg (=64)           Maximum number of ports for connecting to target application
  --config-file arg (=1)                Use config.ncu-cfg config file to set parameters. Searches in the current 
        

In [ ]:
%%writefile 2Dredmodified.cu

#include <iostream>         // For standard input/output operations
#include <cuda_runtime.h>   // For CUDA runtime API
#include <device_launch_parameters.h> // Optional but ensures threadIdx, blockIdx, etc., are defined. mostly included with cuda_runtime.h
#include <stdlib.h>

#define num_threads 512

__global__ void reduce2Dmod(float* src, float* dst, int rows, int cols){

  int index = threadIdx.x;
  //Each thread mapped to different elements of each row. (memory coalescing)

  for (int i=0; i<rows; i++){
    float sum=0.0f;
    for (int j=index; j<cols; j+=num_threads){
      sum+=src[i*cols+j];
    }
    __shared__ float partial_sums[num_threads];
    partial_sums[index] = sum;

    __syncthreads();

    if (index==0){
    float final=0;
    for (int k=0; k<num_threads; k++){
      final+=partial_sums[k];
    }
    dst[i]= final;
    }
  }
}

int main(){
  int rows=10000, cols=10000;
  size_t size= rows*cols*sizeof(float);
  float* h_src= new float[rows*cols];
  float* h_dst = new float[rows];

  for (int i=0; i<rows*cols; i++){
    h_src[i] = 1.0f;
  }

  float* d_src, *d_dst;
  cudaMalloc(&d_src,size);
  cudaMalloc(&d_dst, rows*sizeof(float));

  cudaMemcpy(d_src,h_src,size,cudaMemcpyHostToDevice);

  reduce2Dmod<<<1,num_threads>>>(d_src,d_dst,rows,cols);

  cudaDeviceSynchronize();

  cudaMemcpy(h_dst,d_dst, rows*sizeof(float),cudaMemcpyDeviceToHost);

  for (int i=0; i<20; i++){
    std::cout << h_dst[i] << " ";
  }

  return 0;


}

Writing 2Dredmodified.cu


In [ ]:
!nvcc  -gencode=arch=compute_75,code=sm_75 -o 2Dmod 2Dredmodified.cu

In [ ]:
!nvprof ./2Dmod

==6760== NVPROF is profiling process 6760, command: ./2Dmod
10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ==6760== Profiling application: ./2Dmod
==6760== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   62.22%  95.397ms         1  95.397ms  95.397ms  95.397ms  [CUDA memcpy HtoD]
                   37.78%  57.929ms         1  57.929ms  57.929ms  57.929ms  reduce2Dmod(float*, float*, int, int)
                    0.00%  5.6000us         1  5.6000us  5.6000us  5.6000us  [CUDA memcpy DtoH]
      API calls:   38.64%  97.039ms         2  48.520ms  83.280us  96.956ms  cudaMalloc
                   38.13%  95.774ms         2  47.887ms  105.58us  95.668ms  cudaMemcpy
                   23.07%  57.947ms         1  57.947ms  57.947ms  57.947ms  cudaDeviceSynchronize
                    0.09%  214.89us         1  214.89us  214.89us  214.89us  cudaLaunchKe

The second kernel (my implementation) is better for larger datasets. Execution time of kernel is lesser.

Given below is the implementation with two dimensions of threads.

In [ ]:
%%writefile 2Dredmod2D.cu

#include <iostream>         // For standard input/output operations
#include <cuda_runtime.h>   // For CUDA runtime API
#include <device_launch_parameters.h> // Optional but ensures threadIdx, blockIdx, etc., are defined. mostly included with cuda_runtime.h
#include <stdlib.h>
#include <cmath>

using namespace std;

#define blocksize 32

//Only one block in the x-direction does the actual summation work for each row.
//Multiple blocks in the y-direction are used to parallelize across rows.
//Only threads in blockIdx.x == 0 are performing all the computation.
// Threads in blocks where blockIdx.x > 0 do nothing at all — they are launched but skip the work due to this condition.
//(to avoid redundant compute and write to the same dst[i])


__global__ void reduce2Dmod(float* src, float* dst, int rows, int cols){

  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = threadIdx.x;
  int row_stride = gridDim.y * blockDim.y;

  __shared__ float partial_sums[blocksize][blocksize];

  for (int i=row; i<rows; i+=row_stride){
    if (blockIdx.x==0){
      float partial=0.0f;
      for (int j=col; j<cols; j+=blockDim.x){
        partial+=src[i*cols + j];
      }

      partial_sums[threadIdx.y][threadIdx.x] = partial;

      __syncthreads();

      if (col==0){
        float final=0.0f;
        for (int k=0; k<blocksize; k++){
           final+=partial_sums[threadIdx.y][k];}


        dst[i]=final;
      }}
    }
  }


int main(){
  int rows=10000, cols=10000;
  size_t size= rows*cols*sizeof(float);
  float* h_src= new float[rows*cols];
  float* h_dst = new float[rows];

  for (int i=0; i<rows*cols; i++){
    h_src[i] = 1.0f;
  }

  int minGridSize, blocksize_opt;
  cudaOccupancyMaxPotentialBlockSize(&minGridSize, &blocksize_opt,reduce2Dmod , 0, 0);
  printf("Recommended grid size: %d, block size: %d\n", minGridSize, blocksize_opt);

  float* d_src, *d_dst;
  cudaMalloc(&d_src,size);
  cudaMalloc(&d_dst, rows*sizeof(float));

  cudaMemcpy(d_src,h_src,size,cudaMemcpyHostToDevice);

  dim3 blockSize(32,32);
  //dim3 gridSize(ceil(rows/blocksize),ceil(cols/blocksize));
  dim3 gridSize(1,2);

  reduce2Dmod<<<gridSize, blockSize>>>(d_src,d_dst,rows,cols);

  cudaDeviceSynchronize();

  cudaMemcpy(h_dst,d_dst, rows*sizeof(float),cudaMemcpyDeviceToHost);

  float maxerror=0;
  for (int i=0; i<rows; i++){
    maxerror= max(maxerror,abs(h_dst[i]-rows));
  }
  cout << maxerror<< endl;

  for (int i=0; i<10; i++){
    cout << h_dst[i] << " ";
  }


  for (int i=rows-1; i>rows-10; i--){
    cout << h_dst[i] << " ";
  }
  cout << "hey" <<endl;
  return 0;


}

Overwriting 2Dredmod2D.cu


In [ ]:
!nvcc -gencode=arch=compute_75,code=sm_75 -o 2Dmod2D 2Dredmod2D.cu

In [ ]:
!nvprof ./2Dmod2D

==1302== NVPROF is profiling process 1302, command: ./2Dmod2D
Recommended grid size: 40, block size: 1024
0
10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 hey
==1302== Profiling application: ./2Dmod2D
==1302== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   74.04%  84.955ms         1  84.955ms  84.955ms  84.955ms  [CUDA memcpy HtoD]
                   25.96%  29.783ms         1  29.783ms  29.783ms  29.783ms  reduce2Dmod(float*, float*, int, int)
                    0.00%  5.4400us         1  5.4400us  5.4400us  5.4400us  [CUDA memcpy DtoH]
      API calls:   65.64%  227.82ms         1  227.82ms  227.82ms  227.82ms  cudaFuncGetAttributes
                   24.75%  85.894ms         2  42.947ms  90.906us  85.803ms  cudaMemcpy
                    8.59%  29.805ms         1  29.805ms  29.805ms  29.805ms  cudaDeviceSynchronize
                    0.34%  1.1

Given below is tree-based reduction

In [ ]:
%%writefile 2Dredtree.cu

#include <iostream>         // For standard input/output operations
#include <cuda_runtime.h>   // For CUDA runtime API
#include <device_launch_parameters.h> // Optional but ensures threadIdx, blockIdx, etc., are defined. mostly included with cuda_runtime.h
#include <stdlib.h>

#define num_threads 512

__global__ void reduce2Dmodtree(float* src, float* dst, int rows, int cols){

  int index = threadIdx.x;

  for (int i=0; i<rows; i++){
    float sum=0.0f;
    for (int j=index; j<cols; j+=num_threads){
      sum+=src[i*cols+j];
    }
    __shared__ float partial_sums[num_threads];
    partial_sums[index] = sum;

    __syncthreads();

   //tree based reduction -- not just 1 thread is doing the work of summing up all the elements of partial_sum array.
   for (int s = 1; s<num_threads; s*=2){
    if (index % (s*2)==0){
      partial_sums[index]+=partial_sums[index+s];
    }
    __syncthreads();
   }
   if (index==0){
   dst[i]=partial_sums[0];}

  }
}

int main(){
  int rows=10000, cols=10000;
  size_t size= rows*cols*sizeof(float);
  float* h_src= new float[rows*cols];
  float* h_dst = new float[rows];

  for (int i=0; i<rows*cols; i++){
    h_src[i] = 1.0f;
  }

  float* d_src, *d_dst;
  cudaMalloc(&d_src,size);
  cudaMalloc(&d_dst, rows*sizeof(float));

  cudaMemcpy(d_src,h_src,size,cudaMemcpyHostToDevice);

  reduce2Dmodtree<<<1,num_threads>>>(d_src,d_dst,rows,cols);

  cudaDeviceSynchronize();

  cudaMemcpy(h_dst,d_dst, rows*sizeof(float),cudaMemcpyDeviceToHost);

  for (int i=0; i<20; i++){
    std::cout << h_dst[i] << " ";
  }

  return 0;


}

Overwriting 2Dredtree.cu


In [ ]:
!nvcc -gencode=arch=compute_75,code=sm_75 -o 2Dmodtree 2Dredtree.cu

In [ ]:
!nvprof ./2Dmodtree

==9872== NVPROF is profiling process 9872, command: ./2Dmodtree
10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ==9872== Profiling application: ./2Dmodtree
==9872== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   57.88%  85.340ms         1  85.340ms  85.340ms  85.340ms  [CUDA memcpy HtoD]
                   42.11%  62.090ms         1  62.090ms  62.090ms  62.090ms  reduce2Dmodtree(float*, float*, int, int)
                    0.00%  5.6320us         1  5.6320us  5.6320us  5.6320us  [CUDA memcpy DtoH]
      API calls:   39.85%  98.143ms         2  49.071ms  80.160us  98.063ms  cudaMalloc
                   34.77%  85.627ms         2  42.814ms  70.872us  85.557ms  cudaMemcpy
                   25.21%  62.096ms         1  62.096ms  62.096ms  62.096ms  cudaDeviceSynchronize
                    0.10%  249.90us         1  249.90us  249.90us  249.90us  

Given below is another implementation of tree-based reduction.

In [ ]:
%%writefile 2Dredtree1.cu

#include <iostream>         // For standard input/output operations
#include <cuda_runtime.h>   // For CUDA runtime API
#include <device_launch_parameters.h> // Optional but ensures threadIdx, blockIdx, etc., are defined. mostly included with cuda_runtime.h
#include <stdlib.h>

#define num_threads 512

__global__ void reduce2Dmodtree1(float* src, float* dst, int rows, int cols){

  int index = threadIdx.x;

  for (int i=0; i<rows; i++){
    float sum=0.0f;
    for (int j=index; j<cols; j+=num_threads){
      sum+=src[i*cols+j];
    }
    __shared__ float partial_sums[num_threads];
    partial_sums[index] = sum;

    __syncthreads();

   //tree based reduction -- not just 1 thread is doing the work of summing up all the elements of partial_sum array.

   for (int s=num_threads/2; s>0; s>>=1) {
    if (index<s){
      partial_sums[index]+=partial_sums[index+s];
    }
    __syncthreads();

   }


   if (index==0){
   dst[i]=partial_sums[0];}

  }
}

int main(){
  int rows=10000, cols=10000;
  size_t size= rows*cols*sizeof(float);
  float* h_src= new float[rows*cols];
  float* h_dst = new float[rows];

  for (int i=0; i<rows*cols; i++){
    h_src[i] = 1.0f;
  }

  float* d_src, *d_dst;
  cudaMalloc(&d_src,size);
  cudaMalloc(&d_dst, rows*sizeof(float));

  cudaMemcpy(d_src,h_src,size,cudaMemcpyHostToDevice);

  reduce2Dmodtree1<<<1,num_threads>>>(d_src,d_dst,rows,cols);

  cudaDeviceSynchronize();

  cudaMemcpy(h_dst,d_dst, rows*sizeof(float),cudaMemcpyDeviceToHost);

  for (int i=0; i<20; i++){
    std::cout << h_dst[i] << " ";
  }

  return 0;


}

Overwriting 2Dredtree1.cu


In [ ]:
!nvcc -gencode=arch=compute_75,code=sm_75 -o 2Dmodtree1 2Dredtree1.cu

In [ ]:
!nvprof ./2Dmodtree1

==12007== NVPROF is profiling process 12007, command: ./2Dmodtree1
10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ==12007== Profiling application: ./2Dmodtree1
==12007== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   59.62%  91.149ms         1  91.149ms  91.149ms  91.149ms  [CUDA memcpy HtoD]
                   40.38%  61.729ms         1  61.729ms  61.729ms  61.729ms  reduce2Dmodtree1(float*, float*, int, int)
                    0.00%  5.6320us         1  5.6320us  5.6320us  5.6320us  [CUDA memcpy DtoH]
      API calls:   40.95%  106.48ms         2  53.240ms  93.401us  106.39ms  cudaMalloc
                   35.17%  91.453ms         2  45.727ms  69.406us  91.384ms  cudaMemcpy
                   23.74%  61.738ms         1  61.738ms  61.738ms  61.738ms  cudaDeviceSynchronize
                    0.07%  178.26us         1  178.26us  178.26us  178

Using warp level reductions


In [ ]:
%%writefile 2Dwarp_red.cu

#include <iostream>         // For standard input/output operations
#include <cuda_runtime.h>   // For CUDA runtime API
#include <device_launch_parameters.h> // Optional but ensures threadIdx, blockIdx, etc., are defined. mostly included with cuda_runtime.h
#include <stdlib.h>

#define num_threads 512

__global__ void reduce2Dwarplevel(float* src, float* dst, int rows, int cols){

  int index = threadIdx.x;
  int warps = num_threads/32;

  for (int i=0; i<rows; i++){
    float sum=0.0f;
    for (int j=index; j<cols; j+=num_threads){
      sum+=src[i*cols+j];
    }
    __shared__ float partial_sums[num_threads/32];

    for (int offset = 16; offset > 0; offset /= 2) {
    sum += __shfl_down_sync(0xFFFFFFFF, sum, offset);
}
    int warp_id = index/32;
    int lane = index % 32;
    if (lane==0) partial_sums[warp_id] = sum;

    __syncthreads();


    //now we need to do inter-warp reduction.
    if (index==0){
      float final =0.0f;
      for (int k=0; k<warps; k++){
        final+=partial_sums[k];
      }
      dst[i]= final;

    }



}}

int main(){
  int rows=10000, cols=10000;
  size_t size= rows*cols*sizeof(float);
  float* h_src= new float[rows*cols];
  float* h_dst = new float[rows];

  for (int i=0; i<rows*cols; i++){
    h_src[i] = 1.0f;
  }

  float* d_src, *d_dst;
  cudaMalloc(&d_src,size);
  cudaMalloc(&d_dst, rows*sizeof(float));

  cudaMemcpy(d_src,h_src,size,cudaMemcpyHostToDevice);

  reduce2Dwarplevel<<<1,num_threads>>>(d_src,d_dst,rows,cols);

  cudaDeviceSynchronize();

  cudaMemcpy(h_dst,d_dst, rows*sizeof(float),cudaMemcpyDeviceToHost);

  for (int i=0; i<20; i++){
    std::cout << h_dst[i] << " ";
  }
  cudaFree(d_dst);
  cudaFree(d_src);
  delete[] h_dst;
  delete[] h_src;

  return 0;


}

Writing 2Dwarp_red.cu


In [ ]:
!nvcc -gencode=arch=compute_75,code=sm_75 -o 2Dmodwarp 2Dwarp_red.cu

In [ ]:
!nvprof ./2Dmodwarp

==1917== NVPROF is profiling process 1917, command: ./2Dmodwarp
10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 10000 ==1917== Profiling application: ./2Dmodwarp
==1917== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   67.57%  94.854ms         1  94.854ms  94.854ms  94.854ms  [CUDA memcpy HtoD]
                   32.42%  45.511ms         1  45.511ms  45.511ms  45.511ms  reduce2Dwarplevel(float*, float*, int, int)
                    0.00%  5.4730us         1  5.4730us  5.4730us  5.4730us  [CUDA memcpy DtoH]
      API calls:   63.41%  249.72ms         2  124.86ms  135.64us  249.58ms  cudaMalloc
                   24.43%  96.209ms         2  48.105ms  76.831us  96.133ms  cudaMemcpy
                   11.56%  45.520ms         1  45.520ms  45.520ms  45.520ms  cudaDeviceSynchronize
                    0.30%  1.1945ms         1  1.1945ms  1.1945ms  1.1945ms